In [1]:
from xgboost import XGBClassifier
import csv
import pandas as pd
train = pd.read_csv("data/hb_train_1.csv", header=None)
val = pd.read_csv("data/hb_val.csv", header=None)
test = pd.read_csv("data/hb_test.csv", header=None)

In [2]:
# y_train = train.iloc[:,0]
# X_train = train.iloc[:, [1,27]]
# y_test = test.iloc[:,0]
# X_test = test.iloc[:, [1,27]]

In [3]:
from xgboost import XGBClassifier
import xgboost as xgb
# param = {'max_depth':2, 'eta':1, 'objective':'binary:logistic' }
# model = xgb.train(param, X_train, y_train)


In [4]:
# xgb_pred = model.predict(X_test)

In [5]:
from sklearn.metrics import classification_report
# print(classification_report(y_test, xgb_pred))


In [6]:
# This function returns the true positive, false positive, true negative, and false negative counts.
def perf_measure(y_actual, y_pred):
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for i in range(len(y_pred)): 
        if y_actual[i]==y_pred[i]==1:
           TP += 1
        if y_pred[i]==1 and y_actual[i]!=y_pred[i]:
           FP += 1
        if y_actual[i]==y_pred[i]==0:
           TN += 1
        if y_pred[i]==0 and y_actual[i]!=y_pred[i]:
           FN += 1

    return(TP, FP, TN, FN)

# perf_measure(y_test, xgb_pred)

In [7]:
from numpy import loadtxt
train = loadtxt("data/hb_train_1.csv", delimiter="," )
val = loadtxt("data/hb_val.csv", delimiter="," )
test = loadtxt("data/hb_test.csv", delimiter=",")

In [8]:
y_train = train[:, 0]
X_train = train[:, 1:27]
y_test = test[:, 0]
X_test = test[:, 1:27]


## XGBoost on one site's data (61% accuracy)

In [9]:
model = XGBClassifier(use_label_encoder=False)
model.fit(X_train, y_train, verbose=2)
# make predictions for test data
xgb_pred = model.predict(X_test)
print(classification_report(y_test, xgb_pred))

/Users/andrewzhang/opt/anaconda3/lib/python3.8/site-packages/xgboost/data.py:104: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


[14:15:42] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
              precision    recall  f1-score   support

         0.0       0.56      0.96      0.71     20032
         1.0       0.87      0.25      0.39     19968

    accuracy                           0.61     40000
   macro avg       0.71      0.61      0.55     40000
weighted avg       0.71      0.61      0.55     40000



In [10]:
perf_measure(y_test, xgb_pred)

(5006, 775, 19257, 14962)

# 2 sites data regular xgboost (71% accuracy)


In [11]:
import numpy as np
input2 = loadtxt("data/hb_train_2.csv", delimiter="," )

train2 = np.concatenate([train, input2])

yt2 = train2[:, 0]
xt2 = train2[:, 1:27]

model = XGBClassifier(use_label_encoder=False)
model.fit(xt2, yt2)
# make predictions for test data
xgb_pred2 = model.predict(X_test)
print(classification_report(y_test, xgb_pred2))
perf_measure(y_test, xgb_pred)

/Users/andrewzhang/opt/anaconda3/lib/python3.8/site-packages/xgboost/data.py:104: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


[14:15:49] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
              precision    recall  f1-score   support

         0.0       0.71      0.71      0.71     20032
         1.0       0.71      0.71      0.71     19968

    accuracy                           0.71     40000
   macro avg       0.71      0.71      0.71     40000
weighted avg       0.71      0.71      0.71     40000



(5006, 775, 19257, 14962)

# Federated: Voting between two sites (70% accuracy)

In [12]:
model1 = XGBClassifier(use_label_encoder=False)
model1.fit(X_train, y_train)
pred1 = model1.predict(X_test)
print(classification_report(y_test, pred1))
print(perf_measure(y_test, pred1))

data2 = loadtxt("data/hb_train_2.csv", delimiter="," )
xt2 = data2[:, 1:27]
yt2 = data2[:, 0]
model2 = XGBClassifier(use_label_encoder=False)
model2.fit(xt2, yt2)
pred2 = model2.predict(X_test)
print(classification_report(y_test, pred2))
print(perf_measure(y_test, pred2))

/Users/andrewzhang/opt/anaconda3/lib/python3.8/site-packages/xgboost/data.py:104: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


[14:16:00] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
              precision    recall  f1-score   support

         0.0       0.56      0.96      0.71     20032
         1.0       0.87      0.25      0.39     19968

    accuracy                           0.61     40000
   macro avg       0.71      0.61      0.55     40000
weighted avg       0.71      0.61      0.55     40000

(5006, 775, 19257, 14962)


/Users/andrewzhang/opt/anaconda3/lib/python3.8/site-packages/xgboost/data.py:104: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


[14:16:08] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
              precision    recall  f1-score   support

         0.0       0.87      0.25      0.38     20032
         1.0       0.56      0.96      0.71     19968

    accuracy                           0.60     40000
   macro avg       0.72      0.60      0.55     40000
weighted avg       0.72      0.60      0.55     40000

(19244, 15106, 4926, 724)


In [13]:
prob1 = model1.predict_proba(X_test)
prob1

array([[0.9758602 , 0.02413985],
       [0.8741944 , 0.1258056 ],
       [0.96343887, 0.0365611 ],
       ...,
       [0.20763099, 0.792369  ],
       [0.69789803, 0.302102  ],
       [0.62504923, 0.3749508 ]], dtype=float32)

In [14]:
prob2 = model2.predict_proba(X_test)
prob2

array([[0.9060826 , 0.09391744],
       [0.3606001 , 0.6393999 ],
       [0.591596  , 0.408404  ],
       ...,
       [0.05627292, 0.9437271 ],
       [0.06083065, 0.93916935],
       [0.1133765 , 0.8866235 ]], dtype=float32)

In [15]:
average = prob1 / 2 + prob2 / 2
avg_pred = []
for ar in average:
    if ar[0] > ar[1]:
        avg_pred.append(0)
    else:
        avg_pred.append(1)
print(classification_report(y_test, avg_pred))
print("TP, FP, TN, FN: ")
print(perf_measure(y_test, avg_pred))

              precision    recall  f1-score   support

         0.0       0.70      0.71      0.70     20032
         1.0       0.70      0.70      0.70     19968

    accuracy                           0.70     40000
   macro avg       0.70      0.70      0.70     40000
weighted avg       0.70      0.70      0.70     40000

TP, FP, TN, FN: 
(13934, 5898, 14134, 6034)


# Federated Voting between 2 sites (logistic reg voting) The log reg part trained with central site data

In [16]:
prob2

array([[0.9060826 , 0.09391744],
       [0.3606001 , 0.6393999 ],
       [0.591596  , 0.408404  ],
       ...,
       [0.05627292, 0.9437271 ],
       [0.06083065, 0.93916935],
       [0.1133765 , 0.8866235 ]], dtype=float32)

In [17]:
prob1

array([[0.9758602 , 0.02413985],
       [0.8741944 , 0.1258056 ],
       [0.96343887, 0.0365611 ],
       ...,
       [0.20763099, 0.792369  ],
       [0.69789803, 0.302102  ],
       [0.62504923, 0.3749508 ]], dtype=float32)

In [18]:
prob_test = []
for i in range(len(prob1)):
    prob_test.append([prob1[i][1], prob2[i][1]])

In [26]:
pred1 = model1.predict_proba(X_train)
pred2 = model2.predict_proba(X_train)
probs = []
for i in range(len(pred1)):
    probs.append([pred1[i][1], pred2[i][1]])


## When testing it on the central site, the results are very good

In [20]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(random_state=0).fit(probs, y_train)
lr_pred = lr.predict(probs)
print(classification_report(y_train, lr_pred))
print("TP, FP, TN, FN: ")
print(perf_measure(y_train, lr_pred))

              precision    recall  f1-score   support

         0.0       0.89      0.96      0.92     64040
         1.0       0.76      0.54      0.63     15960

    accuracy                           0.87     80000
   macro avg       0.83      0.75      0.78     80000
weighted avg       0.87      0.87      0.87     80000

TP, FP, TN, FN: 
(8621, 2762, 61278, 7339)


# But when it is tested on test set, we see that it is overfitting and actually hurting overall effectiveness

In [21]:
lr = LogisticRegression(random_state=0).fit(probs, y_train)
lr_pred = lr.predict(prob_test)
print(classification_report(y_test, lr_pred))
print("TP, FP, TN, FN: ")
print(perf_measure(y_test, lr_pred))

              precision    recall  f1-score   support

         0.0       0.59      0.93      0.72     20032
         1.0       0.83      0.35      0.49     19968

    accuracy                           0.64     40000
   macro avg       0.71      0.64      0.60     40000
weighted avg       0.71      0.64      0.60     40000

TP, FP, TN, FN: 
(6947, 1450, 18582, 13021)


# Saving model and then passing around sites, not parallel.

In [22]:
model1.save_model('model1.model')

In [23]:
incr_model = XGBClassifier(use_label_encoder=False)
incr_model.fit(xt2, yt2, xgb_model='model1.model')
incr_model.save_model('incr_model.model')
incr_pred = incr_model.predict(X_test)
print(classification_report(y_test, incr_pred))
print("TP, FP, TN, FN: ")
print(perf_measure(y_test, incr_pred))

/Users/andrewzhang/opt/anaconda3/lib/python3.8/site-packages/xgboost/data.py:104: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


[14:16:16] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
              precision    recall  f1-score   support

         0.0       0.85      0.26      0.40     20032
         1.0       0.56      0.95      0.71     19968

    accuracy                           0.61     40000
   macro avg       0.70      0.61      0.55     40000
weighted avg       0.70      0.61      0.55     40000

TP, FP, TN, FN: 
(19013, 14780, 5252, 955)


In [24]:
incr_model2 = XGBClassifier(use_label_encoder=False)
incr_model2.fit(X_train, y_train, xgb_model='incr_model.model')

/Users/andrewzhang/opt/anaconda3/lib/python3.8/site-packages/xgboost/data.py:104: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


[14:16:22] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=12, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', use_label_encoder=False,
              validate_parameters=1, verbosity=None)

In [25]:
incr_pred2 = incr_model2.predict(X_test)
print(classification_report(y_test, incr_pred2))
print("TP, FP, TN, FN: ")
print(perf_measure(y_test, incr_pred2))

              precision    recall  f1-score   support

         0.0       0.57      0.95      0.71     20032
         1.0       0.84      0.27      0.41     19968

    accuracy                           0.61     40000
   macro avg       0.71      0.61      0.56     40000
weighted avg       0.71      0.61      0.56     40000

TP, FP, TN, FN: 
(5379, 988, 19044, 14589)
